# LSML2 HSE Final Project

 - **Models Manegement:** MlFlow
 - **Web Interface Manegement:** Streamlit App
 - **ModelAPI**: ML flow serving

 ## Task

 Vehicles Damages Detection. 
 Detect vehicle damages using modern CV models

## Models

Compared Models: Yolo11n, Yolo11s


 

## Service Start With Docker Compose



In [ ]:
%%writefile docker-compose.yml
version: "3.7"

services:
  mlflow:
    build: mlflow
    ports:
      - 5001:5001
    volumes:
      - ./mlflow/data:/mldata
  webapp:
    build: web_app
    ports:
      - 8501:8501


In [2]:
import torch

if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
        
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
device

device(type='mps')

## Dataset

https://universe.roboflow.com/claimoo-52a5r/car-damage-detection-20na7/dataset/6

In [ ]:
## Loading Dataset

from roboflow import Roboflow
rf = Roboflow(api_key="CL8s41JqrawRz5ahTY5P")
project = rf.workspace("claimoo-52a5r").project("car-damage-detection-20na7")
version = project.version(6)
dataset = version.download("yolov11", 'datasets/damage_yolo')

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to dataset in yolov11:: 100%|██████████| 3500/3500 [00:00<00:00, 6962.32it/s]


# ML Flow

In [1]:
import mlflow
MLFLOW_SERVER_URL = 'http://0.0.0.0:5001/'
experiment_name = 'damage_detection_yolo'
client = mlflow.tracking.MlflowClient(MLFLOW_SERVER_URL)

mlflow.set_tracking_uri(MLFLOW_SERVER_URL)
# mlflow.set_experiment(experiment_name)

In [3]:
## Prepare MLFlow Yolo Models Tracking
from ultralytics import settings
import os

# Update a setting
settings.update({"mlflow": True})
# os.environ["MLFLOW_EXPERIMENT_NAME"]=experiment_name

In [35]:
%%writefile damage_dataset.yaml

path: damage_yolo # dataset root dir
train: train # train images (relative to 'path') 4 images
val: valid # val images (relative to 'path') 4 images

# Classes (4 classes)
names:
    0: Dent
    1: Scratch_or_spot
    2: Shatter
    3: Tear

Overwriting damage_dataset.yaml


In [42]:
device

device(type='mps')

In [5]:
# Train model Yolo11n
## Loading Base Models to MLFlow
from ultralytics import YOLO

# Train the model
model = YOLO("damage_detection_yolo/train/weights/last.pt")
results = model.train(
    project=experiment_name, # ML FLow Experiment Name
    name='train', #ML FLow Run Name
    data="damage_dataset.yaml", 
    device=device,
    epochs=2, 
    imgsz=640)


    

engine/trainer: task=detect, mode=train, model=damage_detection_yolo/train/weights/last.pt, data=damage_dataset.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=damage_detection_yolo, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscri

train: Scanning /Users/vanitu/hse/LSML2/final/datasets/damage_yolo/train/labels.cache... 1521 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1521/1521 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 4275, len(boxes) = 6255. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning /Users/vanitu/hse/LSML2/final/datasets/damage_yolo/valid/labels.cache... 140 images, 0 backgrounds, 0 corrupt: 100%|██████████| 140/140 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 153, len(boxes) = 234. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
Plotting labels to damage_detection_yolo/train2/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
MLflow: logging run_id(31721a3ae714436cb317bbdae7ba3ceb) to http://0.0.0.0:5001/
MLflow: disable with 'yolo settings mlflow=False'
MLflow: WARNING ⚠️ Failed to initialize: API request to http://0.0.0.0:5001/api/2.0/mlflow/runs/log-batch failed with exception HTTPConnectionPool(host='0.0.0.0', port=5001): Max retries exceeded with url: /api/2.0/mlflow/runs/log-batch (Caused by ResponseError('too many 500 error responses'))
MLflow: WARNING ⚠️ Not tracking this run
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to damage_detection_yolo/train2
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/2      6.35G      2.644      3.777      2.241        109        640:   5%|▌         | 5/96 [00:30<09:17,  6.13s/it]


KeyboardInterrupt: 